# Advanced Tutorial 4: Trace

## Overview
In this tutorial, we will discuss:
* [Customizing Traces](#ta04customize)
    * [Example](#ta04example)
* [More About Traces](#ta04more)
    * [Inputs, Outputs, and Mode](#ta04iom)
    * [Data](#ta04data)
    * [System](#ta04system)
* [Trace Communication](#ta04communication)
* [Other Trace Usages](#ta04other)
    * [Debugging/Monitoring](#ta04debug)
* [Related Apphub Examples](#ta04apphub)

Let's create a function to generate a pipeline, model and network to be used for the tutorial:

In [1]:
import fastestimator as fe
from fastestimator.architecture.tensorflow import LeNet
from fastestimator.dataset.data import mnist
from fastestimator.op.numpyop.univariate import ExpandDims, Minmax
from fastestimator.op.tensorop.loss import CrossEntropy
from fastestimator.op.tensorop.model import ModelOp, UpdateOp


def get_pipeline_model_network(model_name="LeNet", batch_size=32):
    train_data, eval_data = mnist.load_data()
    test_data = eval_data.split(0.5)
    
    pipeline = fe.Pipeline(train_data=train_data,
                           eval_data=eval_data,
                           test_data=test_data,
                           batch_size=batch_size,
                           ops=[ExpandDims(inputs="x", outputs="x"), 
                                Minmax(inputs="x", outputs="x")])

    model = fe.build(model_fn=LeNet, optimizer_fn="adam", model_name=model_name)

    network = fe.Network(ops=[
        ModelOp(model=model, inputs="x", outputs="y_pred"),
        CrossEntropy(inputs=("y_pred", "y"), outputs="ce"),
        UpdateOp(model=model, loss_name="ce")
    ])

    return pipeline, model, network

<a id='ta04customize'></a>

## Customizing Traces
In [Beginner Tutorial 7](../beginner/t07_estimator.ipynb), we talked about the basic concept and structure of `Traces` and used a few `Traces` provided by FastEstimator. We can also customize a Trace to suit our needs. Let's look at an example of a custom trace implementation:

<a id='ta04example'></a>

### Example
We can utilize traces to calculate any custom metric needed for monitoring or controlling training. Below, we implement a trace for calculating the F-beta score of our model.

In [2]:
from fastestimator.util import to_number
from fastestimator.trace import Trace
from sklearn.metrics import fbeta_score
import numpy as np

class FBetaScore(Trace):
    def __init__(self, true_key, pred_key, beta=2, output_name="f_beta_score", mode=["eval", "test"]):
        super().__init__(inputs=(true_key, pred_key), outputs=output_name, mode=mode)
        self.true_key = true_key
        self.pred_key = pred_key
        self.beta = beta
        self.y_true = []
        self.y_pred = []
        
    def on_epoch_begin(self, data):
        self.y_true = []
        self.y_pred = []
        
    def on_batch_end(self, data):
        y_true, y_pred = to_number(data[self.true_key]), to_number(data[self.pred_key])
        y_pred = np.argmax(y_pred, axis=-1)
        self.y_pred.extend(y_pred.ravel())
        self.y_true.extend(y_true.ravel())
        
    def on_epoch_end(self, data):
        score = fbeta_score(self.y_true, self.y_pred, beta=self.beta, average="weighted")
        data.write_with_log(self.outputs[0], score)

Now let's calculate the f2-score using our custom `Trace`. f2-score gives more importance to recall.

In [3]:
pipeline, model, network = get_pipeline_model_network()

traces = FBetaScore(true_key="y", pred_key="y_pred", beta=2, output_name="f2_score", mode="eval")
estimator = fe.Estimator(pipeline=pipeline, network=network, epochs=4, traces=traces, log_steps=1000)

estimator.fit()

    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator-Warn: No ModelSaver Trace detected. Models will not be saved.
FastEstimator-Start: step: 1; num_device: 1; logging_interval: 1000; 
FastEstimator-Train: step: 1; ce: 2.3083596; 
FastEstimator-Train: step: 1000; ce: 0.16284753; steps/sec: 656.26; 
FastEstimator-Train: step: 1875; epoch: 1; epoch_time: 3.55 sec; 
FastEstimator-Eval: step: 1875; epoch: 1; ce: 0.035797507; f2_score: 0.9885909522565743; 
FastEstimator-Train: step: 2000; ce: 0.020546585; steps/sec: 615.78; 
FastEstimator-Train: step: 3000; ce: 0.0059753414; steps/sec: 713.25; 
Fas

<a id='ta04more'></a>

## More About Traces
As we have now seen a custom Trace implementaion, let's delve deeper into the structure of `Traces`.

<a id='ta04iom'></a>

### Inputs, Outputs, and Mode
These Trace arguments are similar to the Operator. To recap, the keys from the data dictionary which are required by the Trace can be specified using the `inputs` argument. The `outputs` argument is used to specify the keys which the Trace wants to write into the system buffer. Unlike with Ops, the Trace `inputs` and `outputs` are essentially on an honor system. FastEstimator will not check whether a Trace is really only reading values listed in its `inputs` and writing values listed in its `outputs`. If you are developing a new `Trace` and want your code to work well with the features provided by FastEstimator, it is important to use these fields correctly. The `mode` argument is used to specify the mode(s) for trace execution as with `Ops`. 

<a id='ta04data'></a>

### Data
Through its data argument, Trace has access to the current data dictionary. You can use any keys which the Trace declared as its `inputs` to access information from the data dictionary. You can write the outputs into the `Data` dictionary with or without logging using the `write_with_log` and `write_without_log` methods respectively.

<a id='ta04system'></a>

### System

Traces have access to the current `System` instance which has information about the `Network` and training process. The information contained in `System` is listed below:
* global_step
* num_devices
* log_steps
* total_epochs
* epoch_idx
* batch_idx
* stop_training
* network
* train_steps_per_epoch
* eval_steps_per_epoch
* summary
* experiment_time

We will showcase `System` usage in the [other trace usages](#ta04other) section of this tutorial. 

<a id='ta04communication'></a>

## Trace Communication
We can have multiple traces in a network where the output of one trace is utilized as an input for another, as depicted below: 

<p align="center">
<img src="https://github.com/fastestimator-util/fastestimator-misc/blob/master/resource/pictures/tutorial/t04_advanced_trace_communication.png?raw=true" alt="drawing" width="500"/>
</p>

Let's see an example where we utilize the outputs of the `Precision` and `Recall` `Traces` to generate f1-score:

In [4]:
from fastestimator.trace.metric import Precision, Recall

class CustomF1Score(Trace):
    def __init__(self, precision_key, recall_key, mode=["eval", "test"], output_name="f1_score"):
        super().__init__(inputs=(precision_key, recall_key), outputs=output_name, mode=mode)
        self.precision_key = precision_key
        self.recall_key = recall_key
        
    def on_epoch_end(self, data):
        precision = data[self.precision_key]
        recall = data[self.recall_key]
        score = 2*(precision*recall)/(precision+recall)
        data.write_with_log(self.outputs[0], score)
        

pipeline, model, network = get_pipeline_model_network()

traces = [
    Precision(true_key="y", pred_key="y_pred", mode=["eval", "test"], output_name="precision"),
    Recall(true_key="y", pred_key="y_pred", mode=["eval", "test"], output_name="recall"),
    CustomF1Score(precision_key="precision", recall_key="recall", mode=["eval", "test"], output_name="f1_score")
]
estimator = fe.Estimator(pipeline=pipeline, network=network, epochs=2, traces=traces, log_steps=1000)

In [5]:
estimator.fit()

    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator-Warn: No ModelSaver Trace detected. Models will not be saved.
FastEstimator-Start: step: 1; num_device: 1; logging_interval: 1000; 
FastEstimator-Train: step: 1; ce: 2.305337; 
FastEstimator-Train: step: 1000; ce: 0.024452677; steps/sec: 734.32; 
FastEstimator-Train: step: 1875; epoch: 1; epoch_time: 2.76 sec; 
FastEstimator-Eval: step: 1875; epoch: 1; ce: 0.0569705; 
precision:
[0.97585513,0.98211091,0.9752381 ,0.98080614,0.99562363,0.96210526,
 1.        ,0.98137803,1.        ,0.97504798];
recall:
[0.99589322,1.        ,0.99224806,0.992233

`Note:` precision, recall, and f1-score are displayed for each class

<a id='ta04other'></a>

## Other Trace Usages 

<a id='ta04debug'></a>

### Debugging/Monitoring
Lets implement a custom trace to monitor a model's predictions. Using this, any discrepancy from the expected behavior can be checked and the relevant corrections can be made: 

In [6]:
class MonitorPred(Trace):
    def __init__(self, true_key, pred_key, mode="train"):
        super().__init__(inputs=(true_key, pred_key), mode=mode)
        self.true_key = true_key
        self.pred_key = pred_key
        
    def on_batch_end(self, data):
        print("Global Step Index: ", self.system.global_step)
        print("Batch Index: ", self.system.batch_idx)
        print("Epoch: ", self.system.epoch_idx)
        print("Batch data has following keys: ", list(data.keys()))
        print("Batch true labels: ", data[self.true_key])
        print("Batch predictictions: ", data[self.pred_key])

pipeline, model, network = get_pipeline_model_network(batch_size=4)

traces = MonitorPred(true_key="y", pred_key="y_pred")
estimator = fe.Estimator(pipeline=pipeline, network=network, epochs=2, traces=traces, train_steps_per_epoch=2, log_steps=None)

In [7]:
estimator.fit()

    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator-Warn: No ModelSaver Trace detected. Models will not be saved.
Global Step Index:  1
Batch Index:  1
Epoch:  1
Batch data has following keys:  ['y', 'ce', 'x', 'y_pred']
Batch true labels:  [1 5 8 5]
Batch predictictions:  [[0.09878654 0.11280762 0.10882236 0.0953772  0.09711165 0.09277759
  0.09783419 0.09401798 0.10111833 0.10134653]
 [0.10425894 0.11605782 0.11004242 0.09267453 0.08793817 0.09537386
  0.10757758 0.08135056 0.09903805 0.10568804]
 [0.1016297  0.11371672 0.10940187 0.09458858 0.09116017 0.09185343
  0.10174091 0.08704273 0.1

As you can see, we can visualize information like the global step, batch number, epoch, keys in the data dictionary, true labels, and predictions at batch level using our `Trace`.

<a id='ta04apphub'></a>

## Apphub Examples
You can find some practical examples of the concepts described here in the following FastEstimator Apphubs:

* [CIFAR10](../../apphub/image_classification/cifar10_fast/cifar10_fast.ipynb)